In [1]:
%load_ext Cython

In [2]:
from __future__ import division, absolute_import, print_function
import numpy as np
import dmft.hirschfye as hf
import dmft.common as gf

In [3]:
def test_hf_fast_updatecond(mc, chempot=0, u_int=2, beta=16.,
                            n_tau=2**11, n_matsubara=64):
    parms = {'BETA': beta, 'N_TAU': n_tau, 'N_MATSUBARA': n_matsubara,
             'MU': chempot, 'U': u_int, 'dtau_mc': 0.125, 'n_tau_mc':    mc, }
    tau, w_n, g0t, __, v = hf.setup_PM_sim(parms)

    g0ttp = hf.ret_weiss(g0t)
    kroneker = np.eye(v.size)

    groot = hf.gnewclean(g0ttp, v, 1, kroneker)
    flip = 5
    v[flip] *= -1

    g_flip = hf.gnewclean(g0ttp, v, 1, kroneker)
    g_fast_flip = np.copy(groot)
    hf.gnew(g_fast_flip, v[flip], flip, 1)

    print( np.allclose(g_flip, g_fast_flip))

    g_ffast_flip = np.copy(groot)
    cygnew(g_ffast_flip, v[flip], flip, 1)
    print( np.allclose(g_flip, g_ffast_flip))
    return groot, v

In [4]:
%%cython  --annotate -lcblas
cdef extern from "cblas.h":
    enum CBLAS_ORDER: CblasRowMajor, CblasColMajor
    void lib_dger "cblas_dger"(CBLAS_ORDER Order, int M, int N, double alpha,
                                double *x, int dx, double *y, int dy,
                                double *A, int lda)

cimport numpy as np
import cython
from libc.math cimport exp

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def cygnew(np.ndarray[np.float64_t, ndim=2] g, double v, int k, double sign):
    cdef double dv, ee, alpha
    cdef int N = g.shape[0]
    dv = sign*v*2
    ee = exp(dv)-1.
    alpha = ee/(1. + (1.-g[k, k])*ee)
    cdef np.ndarray[np.float64_t, ndim=1] x = g[:, k].copy()
    cdef np.ndarray[np.float64_t, ndim=1] y = g[k, :].copy()

    x[k] -= 1.
    lib_dger(CblasColMajor, N, N, alpha,
            &x[0], 1, &y[0], 1, &g[0,0], N)

In [5]:
groot,v = test_hf_fast_updatecond(256)
flip=50
a=np.copy(groot)
b=np.copy(groot)
%timeit hf.gnew(b, v[flip], flip, 1)
%timeit cygnew(a, v[flip], flip, 1)

True
True
10000 loops, best of 3: 45.7 µs per loop
10000 loops, best of 3: 37 µs per loop


In [6]:
%%cython  --annotate -lcblas -lgsl
cdef extern from "cblas.h":
    enum CBLAS_ORDER: CblasRowMajor, CblasColMajor
    void lib_dger "cblas_dger"(CBLAS_ORDER Order, int M, int N, double alpha,
                                double *x, int dx, double *y, int dy,
                                double *A, int lda)
cimport numpy as np
import cython
from libc.math cimport exp

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cpdef cygnew(np.ndarray[np.float64_t, ndim=2] g, double v, int k, double sign):
    cdef double dv, ee, alpha
    cdef int N = g.shape[0]
    dv = sign*v*2
    ee = exp(dv)-1.
    alpha = ee/(1. + (1.-g[k, k])*ee)
    cdef np.ndarray[np.float64_t, ndim=1] x = g[:, k].copy()
    cdef np.ndarray[np.float64_t, ndim=1] y = g[k, :].copy()

    x[k] -= 1.
    lib_dger(CblasColMajor, N, N, alpha,
            &x[0], 1, &y[0], 1, &g[0,0], N)


cdef extern from "gsl/gsl_rng.h":
    ctypedef struct gsl_rng_type:
        pass
    ctypedef struct gsl_rng:
        pass
    gsl_rng_type *gsl_rng_mt19937
    gsl_rng *gsl_rng_alloc(gsl_rng_type * T)
    double uniform "gsl_rng_uniform"(gsl_rng *r)
    
cdef gsl_rng *r = gsl_rng_alloc(gsl_rng_mt19937)

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def cyupdate(np.ndarray[np.float64_t, ndim=2] gup,
             np.ndarray[np.float64_t, ndim=2] gdw,
             np.ndarray[np.float64_t, ndim=1] v):
    cdef double dv, ratup, ratdw, rat
    cdef int j, i, up, dw, pair, N=v.shape[0]
    for j in range(N):
        dv = 2.*v[j]
        ratup = 1. + (1. - gup[j, j])*(exp(-dv)-1.)
        ratdw = 1. + (1. - gdw[j, j])*(exp( dv)-1.)
        rat = ratup * ratdw
        rat = rat/(1.+rat)
        if rat > uniform(r):
            v[j] *= -1.
            cygnew(gup, v[j], j, 1.)
            cygnew(gdw, v[j], j, -1.)

In [7]:
groot,v = test_hf_fast_updatecond(1024)
gtu, gtd = np.copy(groot), np.copy(groot)
%timeit hf.update(gtu, gtd, v)
groot,v = test_hf_fast_updatecond(1024)
gtu, gtd = np.copy(groot), np.copy(groot)
%timeit cyupdate(gtu, gtd, v)

True
True
1 loops, best of 3: 1.04 s per loop
True
True
1 loops, best of 3: 974 ms per loop


In [8]:
groot,v = test_hf_fast_updatecond(512)
gtu, gtd = np.copy(groot), np.copy(groot)
%timeit hf.update(gtu, gtd, v)
groot,v = test_hf_fast_updatecond(512)
gtu, gtd = np.copy(groot), np.copy(groot)
%timeit cyupdate(gtu, gtd, v)

True
True
10 loops, best of 3: 97.6 ms per loop
True
True
10 loops, best of 3: 108 ms per loop


In [9]:
groot,v = test_hf_fast_updatecond(256)
gtu, gtd = np.copy(groot), np.copy(groot)
%timeit hf.update(gtu, gtd, v)
groot,v = test_hf_fast_updatecond(256)
gtu, gtd = np.copy(groot), np.copy(groot)
%timeit cyupdate(gtu, gtd, v)

True
True
100 loops, best of 3: 12.2 ms per loop
True
True
100 loops, best of 3: 8.37 ms per loop
